We will first start with the necessary imports. For this project, we will need NLTK (for nlp), Gensim (for Word2Vec), SkLearn, Pandas, and Numby (for data structures and processing).

In [1]:
import os;
import re;
import nltk.data;
import logging;
from gensim.models import word2vec;
from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree;
import pandas as pd;
import numpy as np;
import sqlite3;

C:\Users\chawlar\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


From NLTK, we need to download the package "Punkt", which contains a module for obtaining sentences from a text. The package needs to be downloaded first.

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chawlar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
sql_con = sqlite3.connect('input/reddit-comments-may-2015/database.sqlite');

In [4]:
row_limit = 10000000;

In [5]:
sql_data = pd.read_sql("SELECT body FROM May2015 LIMIT " + str(row_limit), sql_con);

In [6]:
comments_arr = [sql_data.iloc[idx]['body'] for idx in range(len(sql_data))];

In [7]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle');

In [8]:
def clean_text(content_arr):
    cleaned_arr = [];
    
    for pos, val in enumerate(content_arr):
        no_tabs = str(val).replace('\t', ' ').replace('\n', '');
        
        correct_apos = re.sub("\’", "\'", no_tabs);
        alphas_only = re.sub("[^a-zA-Z\'\.]", " ", correct_apos);
        multi_spaces = re.sub(" +", " ", alphas_only);
        
        clean_text = multi_spaces.strip();
        
        sentences = tokenizer.tokenize(clean_text);
        sentences = [re.sub("[\.]", "", sentence).split() for sentence in sentences];
        
        if len(clean_text) > 0 and clean_text.count(' ') > 0:
            cleaned_arr.extend(sentences);
        
    return cleaned_arr;

In [9]:
clean_comments = clean_text(comments_arr)

In [10]:
len(clean_comments)

19827235

In [11]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 8       # Number of threads to run in parallel
context = 5          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model

print("Training model...");
model = word2vec.Word2Vec(clean_comments, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling);

# We don't plan on training the model any further, so calling 
# init_sims will make the model more memory-efficient.
model.init_sims(replace=True);

model_name = "100features_40minwords_5context_reddit";
model.save(model_name);

2017-08-31 12:24:46,793 : INFO : collecting all words and their counts
2017-08-31 12:24:46,795 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-08-31 12:24:46,957 : INFO : PROGRESS: at sentence #10000, processed 142673 words, keeping 18815 word types


Training model...


2017-08-31 12:24:47,094 : INFO : PROGRESS: at sentence #20000, processed 287102 words, keeping 29308 word types
2017-08-31 12:24:47,231 : INFO : PROGRESS: at sentence #30000, processed 429872 words, keeping 38124 word types
2017-08-31 12:24:47,386 : INFO : PROGRESS: at sentence #40000, processed 570424 words, keeping 45614 word types
2017-08-31 12:24:47,489 : INFO : PROGRESS: at sentence #50000, processed 709642 words, keeping 52561 word types
2017-08-31 12:24:47,627 : INFO : PROGRESS: at sentence #60000, processed 848223 words, keeping 58451 word types
2017-08-31 12:24:47,760 : INFO : PROGRESS: at sentence #70000, processed 981693 words, keeping 64154 word types
2017-08-31 12:24:47,902 : INFO : PROGRESS: at sentence #80000, processed 1124103 words, keeping 69618 word types
2017-08-31 12:24:48,038 : INFO : PROGRESS: at sentence #90000, processed 1264581 words, keeping 74569 word types
2017-08-31 12:24:48,182 : INFO : PROGRESS: at sentence #100000, processed 1406636 words, keeping 79515

2017-08-31 12:24:53,950 : INFO : PROGRESS: at sentence #740000, processed 10309792 words, keeping 271923 word types
2017-08-31 12:24:54,010 : INFO : PROGRESS: at sentence #750000, processed 10455917 words, keeping 274401 word types
2017-08-31 12:24:54,076 : INFO : PROGRESS: at sentence #760000, processed 10602818 words, keeping 276921 word types
2017-08-31 12:24:54,144 : INFO : PROGRESS: at sentence #770000, processed 10746200 words, keeping 279168 word types
2017-08-31 12:24:54,208 : INFO : PROGRESS: at sentence #780000, processed 10890296 words, keeping 281770 word types
2017-08-31 12:24:54,270 : INFO : PROGRESS: at sentence #790000, processed 11030161 words, keeping 284011 word types
2017-08-31 12:24:54,334 : INFO : PROGRESS: at sentence #800000, processed 11169390 words, keeping 286347 word types
2017-08-31 12:24:54,389 : INFO : PROGRESS: at sentence #810000, processed 11311767 words, keeping 288635 word types
2017-08-31 12:24:54,461 : INFO : PROGRESS: at sentence #820000, processe

2017-08-31 12:24:58,866 : INFO : PROGRESS: at sentence #1450000, processed 21155958 words, keeping 457614 word types
2017-08-31 12:24:58,946 : INFO : PROGRESS: at sentence #1460000, processed 21304060 words, keeping 459902 word types
2017-08-31 12:24:59,022 : INFO : PROGRESS: at sentence #1470000, processed 21461439 words, keeping 462414 word types
2017-08-31 12:24:59,089 : INFO : PROGRESS: at sentence #1480000, processed 21616915 words, keeping 464914 word types
2017-08-31 12:24:59,173 : INFO : PROGRESS: at sentence #1490000, processed 21770992 words, keeping 467229 word types
2017-08-31 12:24:59,232 : INFO : PROGRESS: at sentence #1500000, processed 21922618 words, keeping 469845 word types
2017-08-31 12:24:59,306 : INFO : PROGRESS: at sentence #1510000, processed 22081322 words, keeping 472410 word types
2017-08-31 12:24:59,371 : INFO : PROGRESS: at sentence #1520000, processed 22233405 words, keeping 474622 word types
2017-08-31 12:24:59,438 : INFO : PROGRESS: at sentence #1530000,

...

2017-08-31 12:36:12,008 : INFO : PROGRESS: at 54.52% examples, 1259423 words/s, in_qsize 11, out_qsize 4
2017-08-31 12:36:13,022 : INFO : PROGRESS: at 54.64% examples, 1259799 words/s, in_qsize 12, out_qsize 3
2017-08-31 12:36:14,022 : INFO : PROGRESS: at 54.76% examples, 1260239 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:36:15,038 : INFO : PROGRESS: at 54.88% examples, 1260610 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:36:16,048 : INFO : PROGRESS: at 55.00% examples, 1260961 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:36:17,050 : INFO : PROGRESS: at 55.12% examples, 1261283 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:36:18,050 : INFO : PROGRESS: at 55.24% examples, 1261701 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:36:19,058 : INFO : PROGRESS: at 55.36% examples, 1262084 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:36:20,060 : INFO : PROGRESS: at 55.48% examples, 1262452 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:36:21,061 : INFO : PROGRESS: at 55.59% ex

2017-08-31 12:37:31,507 : INFO : PROGRESS: at 64.32% examples, 1286206 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:37:32,518 : INFO : PROGRESS: at 64.45% examples, 1286580 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:37:33,519 : INFO : PROGRESS: at 64.58% examples, 1286979 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:37:34,527 : INFO : PROGRESS: at 64.71% examples, 1287276 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:37:35,537 : INFO : PROGRESS: at 64.82% examples, 1287389 words/s, in_qsize 12, out_qsize 3
2017-08-31 12:37:36,548 : INFO : PROGRESS: at 64.94% examples, 1287534 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:37:37,560 : INFO : PROGRESS: at 65.06% examples, 1287833 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:37:38,563 : INFO : PROGRESS: at 65.18% examples, 1288072 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:37:39,572 : INFO : PROGRESS: at 65.31% examples, 1288361 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:37:40,584 : INFO : PROGRESS: at 65.44% ex

2017-08-31 12:38:51,077 : INFO : PROGRESS: at 74.06% examples, 1304007 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:38:52,080 : INFO : PROGRESS: at 74.18% examples, 1304183 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:38:53,083 : INFO : PROGRESS: at 74.31% examples, 1304425 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:38:54,100 : INFO : PROGRESS: at 74.43% examples, 1304569 words/s, in_qsize 12, out_qsize 3
2017-08-31 12:38:55,111 : INFO : PROGRESS: at 74.55% examples, 1304794 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:38:56,112 : INFO : PROGRESS: at 74.67% examples, 1305034 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:38:57,125 : INFO : PROGRESS: at 74.79% examples, 1305333 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:38:58,126 : INFO : PROGRESS: at 74.92% examples, 1305588 words/s, in_qsize 16, out_qsize 2
2017-08-31 12:38:59,126 : INFO : PROGRESS: at 75.04% examples, 1305837 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:39:00,130 : INFO : PROGRESS: at 75.15% ex

2017-08-31 12:40:10,663 : INFO : PROGRESS: at 83.85% examples, 1320497 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:40:11,668 : INFO : PROGRESS: at 83.98% examples, 1320666 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:40:12,670 : INFO : PROGRESS: at 84.11% examples, 1320783 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:40:13,690 : INFO : PROGRESS: at 84.24% examples, 1320849 words/s, in_qsize 16, out_qsize 1
2017-08-31 12:40:14,693 : INFO : PROGRESS: at 84.37% examples, 1321077 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:40:15,694 : INFO : PROGRESS: at 84.50% examples, 1321206 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:40:16,696 : INFO : PROGRESS: at 84.62% examples, 1321438 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:40:17,711 : INFO : PROGRESS: at 84.75% examples, 1321685 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:40:18,720 : INFO : PROGRESS: at 84.87% examples, 1321895 words/s, in_qsize 16, out_qsize 1
2017-08-31 12:40:19,722 : INFO : PROGRESS: at 85.00% ex

2017-08-31 12:41:30,174 : INFO : PROGRESS: at 93.78% examples, 1334092 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:41:31,179 : INFO : PROGRESS: at 93.91% examples, 1334271 words/s, in_qsize 16, out_qsize 0
2017-08-31 12:41:32,187 : INFO : PROGRESS: at 94.04% examples, 1334452 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:41:33,202 : INFO : PROGRESS: at 94.17% examples, 1334614 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:41:34,211 : INFO : PROGRESS: at 94.29% examples, 1334538 words/s, in_qsize 12, out_qsize 3
2017-08-31 12:41:35,229 : INFO : PROGRESS: at 94.39% examples, 1334408 words/s, in_qsize 14, out_qsize 1
2017-08-31 12:41:36,230 : INFO : PROGRESS: at 94.49% examples, 1334240 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:41:37,237 : INFO : PROGRESS: at 94.61% examples, 1334306 words/s, in_qsize 15, out_qsize 0
2017-08-31 12:41:38,254 : INFO : PROGRESS: at 94.72% examples, 1334440 words/s, in_qsize 13, out_qsize 2
2017-08-31 12:41:39,265 : INFO : PROGRESS: at 94.85% ex

In [12]:
Z = model.wv.syn0;

In [13]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters);
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering.cluster_centers_, idx;

In [14]:
centers, clusters = clustering_on_wordvecs(Z, 10);

In [15]:
centroid_map = dict(zip(model.wv.index2word, clusters));

In [16]:
'''
Returns a DataFrame of the Closest words to each Cluster center. The closest neighbors are obtained using a \
KDTree, which is trained on the provided Word vectors. The KDTree is then queried for the top 20 words\
which are then used as indices in the Word index representation. The results are added to a dictionary,\
which is used to create and return a DataFrame.

'''
def get_top20_words(index2word, centers, wordvecs):
    tree = KDTree(wordvecs);

    #Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=20) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];

    #Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]

    #A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1

    return df;

In [17]:
model.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])

[('regent', 0.9203996062278748),
 ('queen', 0.8904438018798828),
 ('emperor', 0.8728556632995605),
 ('pope', 0.8679628968238831),
 ('Targaryen', 0.8676916360855103),
 ('duchess', 0.8671002984046936),
 ('Queen', 0.8581911325454712),
 ('monarch', 0.8581573963165283),
 ('heir', 0.853079617023468),
 ('royal', 0.8514959812164307)]

In [18]:
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

In [19]:
model.wv.similar_by_word('queen')

[('princess', 0.8127622008323669),
 ('king', 0.7135525345802307),
 ('lion', 0.7047721147537231),
 ('prince', 0.7032004594802856),
 ('vampire', 0.6717629432678223),
 ('dwarf', 0.6700363159179688),
 ('wolf', 0.6470633149147034),
 ('conqueror', 0.6467578411102295),
 ('pearl', 0.6412856578826904),
 ('sorceress', 0.6389750838279724)]

In [20]:
model.wv.word_vec('amazon')

array([-0.07923435,  0.05483854,  0.01097845,  0.04422838, -0.07037681,
        0.10734435,  0.16134924,  0.12160344,  0.02969441,  0.03430137,
       -0.11568732, -0.12524167, -0.07121836,  0.04380697,  0.1013496 ,
        0.12472779,  0.04589591,  0.13760653,  0.04698094,  0.03496061,
        0.20014355,  0.00040228,  0.13380508,  0.19375399,  0.21526089,
        0.09709407,  0.09275255,  0.04370517,  0.22054465,  0.05111407,
        0.15190397,  0.01703347,  0.11764386,  0.03764349, -0.24294493,
        0.09849081,  0.17770456, -0.03950332, -0.04528545, -0.08315247,
       -0.12004509, -0.02640626, -0.02080552,  0.07661081, -0.0272052 ,
        0.08929471, -0.020763  , -0.05887021,  0.04686536, -0.17885886,
        0.16854522, -0.00171533,  0.02966417,  0.11292598,  0.16883048,
       -0.11760147, -0.0163983 ,  0.05589611, -0.02088387,  0.10412354,
        0.05327627,  0.01598939, -0.17672792, -0.0488485 , -0.00899526,
        0.01144994, -0.04392258, -0.14943314, -0.11008292, -0.04